# **Face Recognition-Based Surveillance System**

# **Project Overview:**

The primary goal of this project is to develop a robust face recognition-based surveillance system that processes live video feeds from CCTV cameras. The system is designed to:

*   Detect faces in real-time.
*   Identify and recognize individuals.
*   Record the names and entry timestamps of recognized individuals into a structured format (CSV/Excel).
*   Ensure high accuracy, scalability, and efficient processing without dropping frames.

# **Motivation and Objectives**

With increasing demand for secure environments, face recognition systems play a pivotal role in automating surveillance tasks. This project aims to:


*   Provide a reliable solution for real-time face detection and recognition.*
*   Enhance the scalability of face recognition systems for large datasets.
*   Compare performance metrics of advanced face recognition models (e.g., DeepFace and FaceNet).
*   Establish a framework that integrates seamlessly with existing CCTV setups.

# **Key Components**

This project is divided into several key steps to ensure a systematic approach and effective implementation. Each step builds upon the previous one, leading to a cohesive and functional system. The steps are as follows:


**1.   Acquire images from CCTV cameras:**

Establish a pipeline to fetch live video feeds from CCTV cameras while ensuring no frames are dropped during processing.

**2.   Develope a face detection model:**

Train and deploy the YOLOv8 model to detect faces in the captured video frames. This step focuses on accurately localizing faces with minimal latency.

**3.   Create Embeddings and recognise faces:**

Use the FaceNet model to generate embeddings for the detected faces.
Employ cosine similarity to compare embeddings with the database and identify individuals.



# **1. Acquire images from CCTV cameras**

We extracted and saved frames from a live CCTV feed using the RTSP protocol. To access the feed, we constructed a stream URL using the provided IP address, port, username, and password. Frames were captured in batches, with each batch containing a specific number of frames, and the frames were saved as image files in a designated directory. This process was repeated for a set number of batches to ensure efficient and organized storage of frames for further use. We also incorporated error handling to address potential issues such as failed stream access or frame capture.

In [ ]:
import cv2
import urllib.parse
import os
from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Directory to save extracted frames
save_dir = "/content/drive/My Drive/LiveFeedFrames"
os.makedirs(save_dir, exist_ok=True)

def save_frames_in_batches(ip, port, username, password, camerapath, save_dir, batch_size=10, total_batches=5):
    username = urllib.parse.quote(username)
    password = urllib.parse.quote(password)

    stream_url = f'rtsp://{username}:{password}@{ip}:{port}/{camerapath}'
    cap = cv2.VideoCapture(stream_url)

    if not cap.isOpened():
        print("Failed to open video stream.")
        return

    batch_count = 0
    while batch_count < total_batches:
        print(f"Saving batch {batch_count + 1}/{total_batches}")
        for i in range(batch_size):
            ret, frame = cap.read()
            if not ret:
                print("Failed to read frame.")
                break

            # Save the frame as an image
            frame_index = batch_count * batch_size + i
            frame_path = os.path.join(save_dir, f"frame_{frame_index:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            print(f"Saved: {frame_path}")

        batch_count += 1
        print(f"Batch {batch_count} saved.")

    cap.release()
    print(f"Saved {batch_count * batch_size} frames in total.")

# Parameters for camera connection
ip = "202.53.85.100"
username="admin"
password = "smart@123"
port="8888"
camerapath="live"

# Extract and save frames in batches
save_frames_in_batches(ip, port, username, password, camerapath, save_dir, batch_size=10, total_batches=10)


# **2. Develope a face detection model**

We trained a custom YOLOv8 model for face detection using a labeled dataset. Since YOLOv8 is a general-purpose object detection model, and our requirement is to detect only faces, we retrained the model using a dataset specifically labeled for faces to create a tailored face detection model. The dataset structure was defined in a YAML configuration file, specifying the paths for training and validation images, the number of classes, and their names. We initialized the YOLOv8 model and trained it for 50 epochs with a batch size of 16 and an image size of 640 pixels, saving the weights and results under a specified name. After training, the model's performance was validated, and it was used to predict and save results for a sample image, ensuring precise face detection.

In [ ]:
!pip install ultralytics==8.0.20

from ultralytics import YOLO

dataset_yaml = """
path: /content/drive/MyDrive/Dataset            # Path to your dataset folder
train: /content/drive/MyDrive/Dataset/train     # Relative path to training images
val: /content/drive/MyDrive/Dataset/val         # Relative path to validation images

# Number of classes and class names
nc: 1
names: ['face']
"""
with open('face_detection.yaml', 'w') as f:
    f.write(dataset_yaml)

# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # Choose appropriate YOLO version

# Train the model with custom callback
model.train(
    data="face_detection.yaml",
    epochs=50,
    batch=16,
    imgsz=640,
    device=0,
    name='yolov8_face_detection' # Name for saving weights and results
)

metrics = model.val()

results = model.predict('/content/Mukesh.jpg', save=True)

print("Model weights saved at:", model.ckpt_path)

# **3. Create Embeddings and Recognise faces**

We created a process to generate and save face embeddings with corresponding labels using a trained YOLOv8 model for face detection and the DeepFace library for embedding generation. The YOLOv8 model detects faces in images from a specified folder, and each detected face is cropped using its bounding box. The cropped face is resized and passed to the DeepFace library to generate embeddings using the FaceNet model. Each embedding is associated with a label, extracted from the image filename, and stored in a dictionary. The annotated images are optionally displayed, and the face database, containing all embeddings and labels, is saved as a .pkl file for future use. This setup enables efficient and reusable face recognition.

In [ ]:
!pip install ultralytics==8.0.20
!pip install deepface

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
from google.colab.patches import cv2_imshow
import pickle  # For saving embeddings and labels

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Dictionary to store embeddings and labels
face_database = {}

# Path to the folder containing images
folder_path = '/content/drive/MyDrive/Test_Data/Train'  # Replace with the path to your folder

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        image_path = os.path.join(folder_path, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Extract the label from the image filename (e.g., "abc.jpg" -> "abc")
        label = os.path.splitext(filename)[0]

        # Predict faces using the trained YOLO model
        results = model.predict(image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = image[int(y1):int(y2), int(x1):int(x2)]

            # Optionally, resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Get the embedding for the cropped face
            embedding = get_face_embedding(face_image)

            # Add the embedding and label to the face database
            face_database[label] = embedding

            # Optionally, print or save the embeddings with labels
            print(f"Processed {label}: Embedding added.")

            # Draw bounding box and label on the image
            cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display the processed image
        #cv2_imshow(image)

# Save the face database for future use
with open("face_database.pkl", "wb") as file:
    pickle.dump(face_database, file)

print("Face database saved successfully!")


To validate and inspect the saved face embeddings, we loaded the previously created pickle file containing the face database. The number of entries (embeddings) in the file was counted and printed, providing an overview of the data stored. Additionally, the labels associated with these embeddings were displayed to verify the correctness and completeness of the saved data. This step ensures that the embedding generation and saving process was successful and that the data is ready for subsequent recognition tasks.

In [ ]:
import pickle

# Load the pickle file
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

# Check the number of entries
print(f"Number of entries in the pickle file: {len(face_database)}")

# Optional: Print the keys (labels) to inspect the data
print("Labels in the pickle file:", list(face_database.keys()))


Here, we implemented a face recognition system by utilizing embeddings stored in a pre-trained face database. Test images were processed to detect faces using a YOLOv8 model trained specifically for face detection. For each detected face, embeddings were generated using the DeepFace library with the FaceNet model. These embeddings were compared against the stored embeddings in the database using cosine similarity.

A label was assigned to each detected face based on the highest similarity score exceeding a defined threshold, ensuring accurate identification. Bounding boxes and labels were drawn on the test images to indicate recognized faces, and the annotated images were saved in an output folder for further analysis. This process enabled us to effectively identify faces in the test dataset.

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from deepface import DeepFace
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load the YOLO model
model = YOLO("/content/drive/MyDrive/YoloV8best.pt")  # Replace with the path to your trained YOLO model

# Load the saved face database
with open("face_database.pkl", "rb") as file:
    face_database = pickle.load(file)

# Function to generate embeddings from the cropped face using DeepFace
def get_face_embedding(face_image):
    embedding = DeepFace.represent(face_image, model_name='Facenet', enforce_detection=False)[0]['embedding']
    return embedding

# Threshold for cosine similarity (tune based on your use case)
SIMILARITY_THRESHOLD = 0.5

# Path to the folder containing test images
test_folder_path = '/content/drive/MyDrive/Test_Data/Test'  # Replace with your folder path

# Loop through all images in the folder
for filename in os.listdir(test_folder_path):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
        test_image_path = os.path.join(test_folder_path, filename)

        # Read the test image
        test_image = cv2.imread(test_image_path)

        # Predict faces using the trained YOLO model
        results = model.predict(test_image_path)

        # Loop through each detected face in the results
        for result in results[0].boxes.data:  # result[0] contains the detection result
            x1, y1, x2, y2 = result[:4]  # Bounding box coordinates (x1, y1, x2, y2)

            # Crop the face from the image using the bounding box
            face_image = test_image[int(y1):int(y2), int(x1):int(x2)]

            # Resize the face image if needed for the model
            face_image = cv2.resize(face_image, (160, 160))  # Example for FaceNet or similar models

            # Get the embedding for the cropped face
            new_embedding = get_face_embedding(face_image)

            # Initialize variables to track the best match
            best_label = "Unknown"
            best_similarity = 0

            # Compare the new embedding with stored embeddings in the face database
            for label, stored_embedding in face_database.items():
                similarity = cosine_similarity([new_embedding], [stored_embedding])[0][0]
                if similarity > best_similarity and similarity > SIMILARITY_THRESHOLD:
                    best_label = label
                    best_similarity = similarity

            # Draw bounding box and label on the image
            cv2.rectangle(test_image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(test_image, best_label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            print(f"Image: {filename}, Detected face is labeled as: {best_label} (similarity: {best_similarity:.2f})")

        # Save or display the image with labels
        output_image_path = os.path.join('/content/verified_images', filename)  # Adjust the output folder as needed
        os.makedirs(os.path.dirname(output_image_path), exist_ok=True)
        cv2.imwrite(output_image_path, test_image)

print("Face recognition completed for all images in the folder.")


# **Enhancing Face Recognition Model to Handle Face Orientations**

# **Objective:**

To improve the face recognition model’s ability to recognize faces captured from various angles and orientations, rather than limiting it to frontal face views.

# **Current Limitation:**

The existing face recognition system effectively detects faces, creates embeddings, and recognizes identities. However, the recognition accuracy drops significantly when a person’s face is oriented away from the frontal view, such as side profiles or tilted angles. This limitation arises due to the model’s training dataset consisting predominantly of frontal face images, which leads to inadequate generalization for non-frontal orientations.

# **Proposed Solution:**

To address this issue, we propose the following enhancements


1.   **Diversify the training DataSet**

      * Include images with varied face orientations in the training dataset, such as:
           * Left and right side profiles.
           *	Tilted faces (upwards and downwards).
           * Rotated faces at various angles.
           *	Slightly occluded faces (e.g., partially covered by hair or accessories).
      *Gather images of individuals from multiple perspectives to ensure comprehensive coverage of possible real-world scenarios.

2.   **Augmentation Techniques**
      
      * Apply data augmentation techniques to artificially create face images with different orientations, including:
           * Rotation at random angles.
           * Flipping (horizontal and vertical).
           * Adding noise, shadows, or blurring to simulate diverse environmental conditions.

3. **Multiview Embedding Creation**

     * Capture embeddings for each individual’s face across multiple orientations.
     * During recognition, compare the test face’s embedding with all stored embeddings (frontal and non-frontal) for the closest match.


# **Outcomes**

Implementing these enhancements will significantly improve the face recognition model’s ability to identify individuals accurately, irrespective of their face orientation. This will make the system more robust and suitable for real-world applications like surveillance, attendance systems, and access control.

# **Next Steps**
   * Collect and curate a diverse dataset with multi-view face images.
   * Implement data augmentation and fine-tuning.
   * Evaluate the model’s performance with an emphasis on recognizing side profiles and tilted faces.
   * Iterate and refine based on testing results.










